# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\tag{1}
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\tag{2}
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otoczenia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\tag{3}
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:
        Input = mat['I_noisy1']

3. Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os
import cv2


if not os.path.exists("MR_data.mat") :
    os.system("wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate")

#TODO Samodzielna
images = loadmat("MR_data.mat")

plt.figure(figsize=(7, 7))
plt.imshow(images["I_noisy1"], 'gray')
plt.xticks([])
plt.yticks([])
plt.show()

### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def gaussian(size, sigma):
    k = size // 2
    x, y = np.mgrid[-k:k+1, -k:k+1]
    g = np.exp(-(x**2 + y**2)/(2*sigma**2))
    return g / g.sum()

def gaussian_bez(size, sigma):
    k = size // 2
    x, y = np.mgrid[-k:k+1, -k:k+1]
    g = np.exp(-(x**2 + y**2)/(2*sigma**2))
    return g


In [ ]:
def gaus(dist, delta_s):
    return np.exp(-(dist**2)/(2*(delta_s)**2))

def euc_dist(P1, P2):
    return np.sqrt((P1[0] - P2[0])**2 + (P1[1] - P2[1])**2)


def p_out(window, filtr, delta_s):
    X, Y = window.shape
    p = 0
    m = [X//2, Y//2]
    
    for i in range(X):
        for j in range(Y):
            XY = [i, j]
            p += gaus(euc_dist(XY, m), delta_s) * window[i][j]
    p /= filtr.sum()
    return p


In [ ]:
#TODO Samodzielna

def classic_conv(I, window, delta_s):
    filtr = gaussian_bez(3, delta_s)
    Iconv = I.copy()
    (X, Y) = Iconv.shape
    half = window // 2
    for i in range(window//2, X-window//2):
        for j in range(window//2, Y-window//2):
            okno = Iconv[i-half:i+half+1, j-half:j+half+1]
            p = p_out(okno, filtr, delta_s)
            Iconv[i][j] = p
    return Iconv

def plot_classic_conv(I, filtr, delta_s):
    R = classic_conv(I, filtr, delta_s)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

    ax1.imshow(I, 'gray')
    ax1.set_yticks([])
    ax1.set_xticks([])
    ax1.set_title("Obraz oryginalny")

    ax2.imshow(R, 'gray')
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_title(f"Obraz dla parametru delta_s = {delta_s}")

    plt.show()


I = images["I_noisy1"]

plot_classic_conv(I, 5, 0.8, 0.6)

In [ ]:
plot_classic_conv(images["I_noisy2"], 7, 0.3, 0.6)

In [ ]:
plot_classic_conv(images["I_noisy3"], 7, 0.3, 20)

### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gaussowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
delta_r = 2

def p_out_biliteral(window, delta_s, delta_r):
    X, Y = window.shape
    p = 0
    normalize = 0
    x = [X//2, Y//2]
    for i in range(X):
        for j in range(Y):
            XY = [i,j]

            g = gaus(euc_dist(x, XY), delta_s)
            diff = np.abs(window[X//2, Y//2] - window[i,j])
            gaus_diff = np.exp(-(diff**2) / (2*(delta_r**2)))

            p += g * gaus_diff*window[i,j]
            normalize += gaus_diff * g
    p /= normalize
    return p


def bilateral(I, window, delta_s, delta_r):
    IBilateral = I.copy()
    (X, Y) = IBilateral.shape
    half = window // 2
    for i in range(window//2, X-window//2):
        for j in range(window//2, Y-window//2):
            okno = IBilateral[i-half:i+half+1, j-half:j+half+1]
            p = p_out_biliteral(okno, delta_s, delta_r)
            IBilateral[i,j] = p
    return IBilateral


def plot_biliteral(I, filtr, delta_s, delta_r):
    R = bilateral(I, filtr, delta_s, delta_r)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

    ax1.imshow(I, 'gray')
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title("Oryginalny obraz")

    ax2.imshow(R, 'gray')
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_title(f"Obraz dla parametru delta_s = {delta_s}, delta_r = {delta_r}")
    plt.show()


I = images["I_noisy1"]

plot_biliteral(I, 5, 2, 20)

In [ ]:
plot_biliteral(images["I_noisy2"], 7, 3, 10)

In [ ]:
plot_biliteral(images["I_noisy3"], 7, 3, 20)